In [1]:
import pandas as pd

In [2]:
def qbratings(year):
    import pandas as pd
    url = f'https://www.pro-football-reference.com/years/{year}/passing.htm'
    df = pd.read_html(url,header=0)[0]
    df = df[df['QBrec'].notna()]
    df = df[['Player','Tm','Pos','Age','G','GS','Cmp','Att','Yds','TD','Int','Rate']]
    df = df[df['Pos'] != 'Pos']
    df = df.replace('\+','',regex=True).astype(object)
    df = df.replace('\*','',regex=True).astype(object)
    df = df.replace('^qb','QB',regex=True).astype(object)
    df = df.replace('^/qb','QB',regex=True).astype(object)
    df = df.replace('^wr','QB',regex=True).astype(object)
    df['Pos'].fillna('QB',inplace=True)
    df['Player'] = df['Player'].str.rstrip()
    columns = df.columns[3:]
    for col in columns:
        df[col] = pd.to_numeric(df[col],errors='coerce')
    df['Year'] = year
    df['Player'] = df.Player.str.replace('[^a-zA-Z]', '', regex = True)
    playerID = []
    for index, row in df.iterrows():
        playerID.append(row['Player'] + str(row['Year']))
    df['PlID'] = playerID
    teamID = []
    for index, row in df.iterrows():
        teamID.append(row['Tm'] + str(row['Year']))
    df['TmID'] = teamID
    return df

In [3]:
def createQBDB(startYear,endYear,function):
    # Import dependencies
    import pandas as pd
    database = pd.DataFrame()
    years = []
    # Iterates through years specified
    for year in range(startYear, endYear + 1):
        years.append(year)
    for year in years:
        database = database.append(function(year))
    return database

((a+b+c+d)/6) * 100
a = ((comp/att) - .3)*5
b = ((yds/att)-3)*.25
c = (td/att)*20
d = 2.375 - ((int/att)*25)

In [4]:
df = createQBDB(2000,2021,qbratings)
df

,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,TD,Int,Rate,Year,PlID,TmID
0,PeytonManning,IND,QB,24,16,16,357,571,4413,33,15,94.7,2000,PeytonManning2000,IND2000
1,JeffGarcia,SFO,QB,30,16,16,355,561,4278,31,10,97.6,2000,JeffGarcia2000,SFO2000
2,ElvisGrbac,KAN,QB,30,15,15,326,547,4169,28,14,89.9,2000,ElvisGrbac2000,KAN2000
3,DaunteCulpepper,MIN,QB,23,16,16,297,474,3937,33,16,98.0,2000,DaunteCulpepper2000,MIN2000
4,BrettFavre,GNB,QB,31,16,16,338,580,3812,20,16,78.0,2000,BrettFavre2000,GNB2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,SeanMannion,MIN,QB,29,1,1,22,36,189,1,0,84.1,2021,SeanMannion2021,MIN2021
61,BrandonAllen,CIN,QB,29,6,1,17,34,149,2,0,81.6,2021,BrandonAllen2021,CIN2021
62,NickMullens,CLE,QB,26,1,1,20,30,147,1,0,89.2,2021,NickMullens2021,CLE2021
63,IanBook,NOR,QB,23,1,1,12,20,135,0,2,40.6,2021,IanBook2021,NOR2021


In [20]:
years = range(2000,2022)
columns = ['Cmp','Att','Yds','TD','Int']
players = []
team = []
cmp = []
att = []
yds = []
td = []
inter = []
rate = []
yrs = []
for index, row in df.iterrows():
    player = row['Player']
    yr = row['Year']
    tm = row['Tm']
    players.append(player)
    yrs.append(yr)
    team.append(tm)
    for year in years:
        if year == yr:
            for column in columns:
                amt = df.loc[(df['Player'] == player) & (df['Year'] <= yr), column].sum()
                if column == 'Cmp':
                    cmp.append(amt)
                elif column == 'Att':
                    att.append(amt)
                elif column == 'Yds':
                    yds.append(amt)
                elif column == 'TD':
                    td.append(amt)
                else:
                    inter.append(amt)
                mean = round(df.loc[(df['Player'] == player) & (df['Year'] <= yr), 'Rate'].mean(),2)
            rate.append(mean)
        else:
            continue
#dfdf = pd.DataFrame({'Player':players,'Year':yrs})
newdf = pd.DataFrame({'Player':players,'Team':team,'Cmp':cmp,'Att':att,'Yds':yds,'TD':td,'Int':inter,'AvgRate':rate,'Year':yrs})

In [21]:
newdf['PassRate'] = ((((((newdf['Cmp']/newdf['Att']) - .3)*5)+(((newdf['Yds']/newdf['Att'])-3)*.25)+((newdf['TD']/newdf['Att'])*20)+(2.375-((newdf['Int']/newdf['Att'])*25)))/6)*100)

playerID = []
for index, row in newdf.iterrows():
    playerID.append(row['Player'] + str(row['Year']))
newdf['PlID'] = playerID


In [22]:
columns = ['Player','Team','Cmp','Att','Yds','TD','Int','AvgRate','PassRate','Year','PlID']
newdf = newdf[columns]
newdf

,Player,Team,Cmp,Att,Yds,TD,Int,AvgRate,PassRate,Year,PlID
0,JakePlummer,ARI,270,475,2946,13,21,66.00,65.995614,2000,JakePlummer2000
1,DaveBrown,ARI,40,69,467,2,3,70.10,70.138889,2000,DaveBrown2000
2,ChrisChandler,ATL,192,331,2236,10,12,73.50,73.533484,2000,ChrisChandler2000
3,DannyKanell,ATL,57,116,524,2,5,49.60,49.640805,2000,DannyKanell2000
4,DougJohnson,ATL,36,67,406,2,3,63.40,63.401741,2000,DougJohnson2000
...,...,...,...,...,...,...,...,...,...,...,...
1208,TomBrady,TAM,7262,11314,84514,624,203,96.92,97.604369,2021,TomBrady2021
1209,RyanTannehill,TEN,2702,4209,30729,199,102,93.23,91.662212,2021,RyanTannehill2021
1210,TaylorHeinicke,WAS,356,551,3739,21,18,73.25,83.291742,2021,TaylorHeinicke2021
1211,GarrettGilbert,WAS,41,69,437,1,1,77.25,76.781401,2021,GarrettGilbert2021


In [24]:
newdf[newdf['Player'] == 'TomBrady']

,Player,Team,Cmp,Att,Yds,TD,Int,AvgRate,PassRate,Year,PlID
88,TomBrady,NWE,264,413,2843,18,12,86.50,86.455811,2001,TomBrady2001
138,TomBrady,NWE,637,1014,6607,46,26,86.10,86.020710,2002,TomBrady2002
198,TomBrady,NWE,954,1541,10227,69,38,86.03,85.976368,2003,TomBrady2003
253,TomBrady,NWE,1242,2015,13919,97,52,87.68,87.523780,2004,TomBrady2004
309,TomBrady,NWE,1576,2545,18029,123,66,88.60,88.509332,2005,TomBrady2005
366,TomBrady,NWE,1895,3061,21558,147,78,88.48,88.408608,2006,TomBrady2006
425,TomBrady,NWE,2293,3639,26364,197,86,92.59,92.978268,2007,TomBrady2007
484,TomBrady,NWE,2300,3650,26440,197,86,91.50,92.950913,2008,TomBrady2008
535,TomBrady,NWE,2671,4215,30838,225,99,92.02,93.382266,2009,TomBrady2009
599,TomBrady,NWE,2995,4707,34738,261,103,93.92,95.222983,2010,TomBrady2010


In [9]:
df = df.sort_values(by = ['Year','Tm'], ascending = [True,True])
df = df[df['Tm'] != '2TM']
df = df[df['Tm'] != '3TM']
df

,Player,Tm,Pos,Age,G,GS,Cmp,Att,Yds,TD,Int,Rate,Year,PlID,TmID
13,JakePlummer,ARI,QB,26,14,14,270,475,2946,13,21,66.0,2000,JakePlummer2000,ARI2000
48,DaveBrown,ARI,QB,30,6,2,40,69,467,2,3,70.1,2000,DaveBrown2000,ARI2000
21,ChrisChandler,ATL,QB,35,14,13,192,331,2236,10,12,73.5,2000,ChrisChandler2000,ATL2000
46,DannyKanell,ATL,QB,27,5,1,57,116,524,2,5,49.6,2000,DannyKanell2000,ATL2000
49,DougJohnson,ATL,QB,23,4,2,36,67,406,2,3,63.4,2000,DougJohnson2000,ATL2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,TomBrady,TAM,QB,44,17,17,485,719,5316,43,12,102.1,2021,TomBrady2021,TAM2021
15,RyanTannehill,TEN,QB,33,17,17,357,531,3734,21,14,89.6,2021,RyanTannehill2021,TEN2021
18,TaylorHeinicke,WAS,QB,28,16,15,321,494,3419,20,15,85.9,2021,TaylorHeinicke2021,WAS2021
58,GarrettGilbert,WAS,QB,30,1,1,20,31,194,0,0,81.9,2021,GarrettGilbert2021,WAS2021


In [11]:
import pandas as pd
lastTeam = 'NaN'
teams = []
yrs = []
tmid = []

for index, row in df.iterrows():
    team = row['Tm']
    year = row['Year']
    teamid = row['TmID']
    if lastTeam != team:
        teams.append(team)
        yrs.append(year)
        tmid.append(teamid)
    else:
        continue
    lastTeam = row['Tm']
        
teamsdf = pd.DataFrame({'Tm':teams,'Year':yrs,'TmID':tmid})
columns = ['Cmp','Att','Yds','TD','Int']
cmp = []
att = []
yds = []
td = []
inter = []
rate = []
for teamid in tmid:
    for column in columns:
        amt = df.loc[(df['TmID'] == teamid), column].sum()
        if column == 'Cmp':
            cmp.append(amt)
        elif column == 'Att':
            att.append(amt)
        elif column == 'Yds':
            yds.append(amt)
        elif column == 'TD':
            td.append(amt)
        else:
            inter.append(amt)
        mean = round(df.loc[df['TmID'] == teamid, 'Rate'].mean(),2)
    rate.append(mean)
            
teamsdf['Cmp'] = cmp
teamsdf['Att'] = att
teamsdf['Yds'] = yds
teamsdf['TD'] = td
teamsdf['Int'] = inter
teamsdf['Rate'] = rate
teamsdf
            
# lastTeam = 'NaN'

# for index,row in qb2000.iterrows():
#     team = row['Tm']
#     cmp = row['Cmp']
#     att = row['Att']
#     yds = row['Yds']
#     td = row['TD']
#     inter = row['Int']
    
        

,Tm,Year,TmID,Cmp,Att,Yds,TD,Int,Rate
0,ARI,2000,ARI2000,310,544,3413,15,24,68.05
1,ATL,2000,ATL2000,285,514,3166,14,20,62.17
2,BAL,2000,BAL2000,284,500,3080,20,19,72.95
3,BUF,2000,BUF2000,307,537,3825,20,10,84.35
4,CAR,2000,CAR2000,324,533,3730,19,18,79.70
...,...,...,...,...,...,...,...,...,...
697,SEA,2021,SEA2021,324,495,3815,30,7,103.05
698,SFO,2021,SFO2021,342,512,4413,25,14,98.00
699,TAM,2021,TAM2021,485,719,5316,43,12,102.10
700,TEN,2021,TEN2021,357,531,3734,21,14,89.60


In [12]:
teamsdf['Tm'].value_counts()

ARI    22
MIA    22
WAS    22
TEN    22
TAM    22
SFO    22
SEA    22
PIT    22
PHI    22
NYJ    22
NYG    22
NWE    22
ATL    22
MIN    22
NOR    22
KAN    22
CLE    22
BAL    22
BUF    22
CAR    22
CHI    22
JAX    22
CIN    22
DAL    22
DEN    22
DET    22
GNB    22
IND    22
OAK    20
HOU    20
SDG    17
STL    16
LAR     6
LAC     5
LVR     2
Name: Tm, dtype: int64

In [25]:
newdf.to_csv('Database_CSVs/passerratings.csv',index=False)

In [ ]:
overallPR = df.groupby('Player').agg({'Cmp':'sum','Att':'sum','Yds':'sum','TD':'sum','Int':'sum','Rate':'mean'})
overallPR['PassRating'] = ((((((overallPR['Cmp']/overallPR['Att']) - .3)*5)+(((overallPR['Yds']/overallPR['Att'])-3)*.25)+((overallPR['TD']/overallPR['Att'])*20)+(2.375-((overallPR['Int']/overallPR['Att'])*25)))/6)*100)
overallPR

In [ ]:
overallPR.sort_values(by='PassRating',ascending=False)

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
overallPR.sort_values(by='PassRating',ascending=False)

In [ ]:
overallPR['Difference'] = overallPR['Rate'] - overallPR['PassRating']
overallPR.sort_values(by='Difference',ascending=False)

In [ ]:
df

In [ ]:
#((((((overallPR['Cmp']/overallPR['Att']) - .3)*5)+(((overallPR['Yds']/overallPR['Att'])-3)*.25)+((overallPR['TD']/overallPR['Att'])*20)+(2.375-((overallPR['Int']/overallPR['Att'])*25)))/6)*100)

In [ ]:
qb2000PR = qb2000.groupby('Player').agg({'Cmp':'sum','Att':'sum','Yds':'sum','TD':'sum','Int':'sum','Rate':'mean'})
qb2001PR = qb2001.groupby('Player').agg({'Cmp':'sum','Att':'sum','Yds':'sum','TD':'sum','Int':'sum','Rate':'mean'})
qb2000PR['PassRating'] = ((((((qb2000PR['Cmp']/qb2000PR['Att']) - .3)*5)+(((qb2000PR['Yds']/qb2000PR['Att'])-3)*.25)+((qb2000PR['TD']/qb2000PR['Att'])*20)+(2.375-((qb2000PR['Int']/qb2000PR['Att'])*25)))/6)*100)
qb2001PR['PassRating'] = ((((((qb2001PR['Cmp']/qb2001PR['Att']) - .3)*5)+(((qb2001PR['Yds']/qb2001PR['Att'])-3)*.25)+((qb2001PR['TD']/qb2001PR['Att'])*20)+(2.375-((qb2001PR['Int']/qb2001PR['Att'])*25)))/6)*100)

In [ ]:
import pandas as pd
years = range(2000,2022)
players = []
yrs = []
tms = []
tmid = []
plid = []
avgrate = []
rating = []
# def passerrating(df):
#     ((((((df['Cmp']/df['Att']) - .3)*5)+
#        (((df['Yds']/df['Att'])-3)*.25)+
#        ((df['TD']/df['Att'])*20)+
#        (2.375-((df['Int']/df['Att'])*25)))/6)*100)
for year in years:
    for index, row in df.iterrows():
        yr = row['Year']
        pl = row['Player']
        if yr == year:
            players.append(pl)
            yrs.append(yr)
            tms.append(row['Tm'])
            tmid.append(row['TmID'])
            plid.append(row['PlID'])
            avgrate.append(df.loc[(df['Player'] == row['Player'])&(df['Year'] <= year)]['Rate'].mean())
            #prDF = df.where(df.Year <= year).groupby(pl).agg({'Cmp':'sum','Att':'sum','Yds':'sum','TD':'sum','Int':'sum'})
            #rating.append(((((((prDF['Cmp']/prDF['Att']) - .3)*5)+(((prDF['Yds']/prDF['Att'])-3)*.25)+((prDF['TD']/prDF['Att'])*20)+(2.375-((prDF['Int']/prDF['Att'])*25)))/6)*100))
newdf = pd.DataFrame({'Player': players, 'Year': yrs,'Team':tms,'AvgRate':avgrate,'TmID':tmid,'PlID':plid})
newdf

In [ ]:
newdf.to_csv('passerratings.csv',index=False)

In [ ]:
pd.set_option('display.max_rows', None)
newdf['TmID'].value_counts(ascending=False)

In [ ]:
newdf[newdf['Player'] == 'TomBrady']

In [ ]:
prDF = df.where(df.Year <= 2001).groupby(df['Player']).agg({'Cmp':'sum','Att':'sum','Yds':'sum','TD':'sum','Int':'sum'})

In [ ]:
prDF